<a href="https://colab.research.google.com/github/NoamTene/GMATcolab/blob/main/Ex_R2020a_GMAT_States.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# State Management with the GMAT API
The state data in GMAT can be a bit confusing.  This notebook introduces the state variables as used for a GMAT Spacecraft, and provides some pointers on the manipulation of the state data.

## Prepare the GMAT Environment
Before the API can be used, it needs to be loaded into the Python system and initialized using a GMAT startup file.  This can be done from the GMAT bin folder by importing the gmatpy module, but using that approach tends to leave files in the bin folder that may annoy other users.  Running from an outside folder takes a few steps, which have been captured in the load_gmat.py file in the GMAT api folder.  After preparing the API for use (see the API "read me" file, api/API_README.txt), copy load_gmat.py into the folder you are using for Jupyter notebooks.  Then import it:

In [9]:
!wget https://yer.dl.sourceforge.net/project/gmat/GMAT/GMAT-R2022a/gmat-ubuntu-x64-R2022a.tar.gz
!tar -xf gmat-ubuntu-x64-R2022a.tar.gz
!python "GMAT/R2022a/api/BuildApiStartupFile.py"
!sed -i 's/<TopLevelGMATFolder>/\/content\/GMAT\/R2022a/' GMAT/R2022a/api/load_gmat.py
%cd GMAT/R2022a/api

--2023-08-26 22:30:17--  https://yer.dl.sourceforge.net/project/gmat/GMAT/GMAT-R2022a/gmat-ubuntu-x64-R2022a.tar.gz
Resolving yer.dl.sourceforge.net (yer.dl.sourceforge.net)... 94.20.154.18, 2a01:6480:20::5
Connecting to yer.dl.sourceforge.net (yer.dl.sourceforge.net)|94.20.154.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390518023 (372M) [application/x-gzip]
Saving to: ‘gmat-ubuntu-x64-R2022a.tar.gz’

gmat-ubuntu-x64-R20 100%[===================>] 372.43M  16.9MB/s    in 22s     

2023-08-26 22:30:40 (16.7 MB/s) - ‘gmat-ubuntu-x64-R2022a.tar.gz’ saved [390518023/390518023]

/content/GMAT/R2022a/api/GMAT/R2022a/api


In [10]:
from load_gmat import *

## Configure a Spacecraft
We'll need an object that provides the state.  Here's a basic spacecraft, along with a reference to the state data inside of the spacecraft:

In [11]:
sat = gmat.Construct("Spacecraft","MySat")
iState = sat.GetState()

The state reference here, iState, operates on the member of the Spacecraft object that GMAT uses when running a simulation.  The "internal state," referenced by iState here, is the Earth-centered mean-of-J2000 equatorial representation of position and velocity of the spacecraft MySat.  The data is contained in a GmatState object:

In [12]:
iState

<gmat_py.GmatState; proxy of <Swig Object of type 'GmatState *' at 0x7d27001b11a0> >

GmatState objects are used to collect together an epoch and a vector of data.  These data can be accessed directly:

In [13]:
print("The state epoch is ", iState.GetEpoch(), ", the state has ", iState.GetSize(),
      " elements, and contains the data ", iState.GetState())

The state epoch is  21545.000000397937 , the state has  6  elements, and contains the data  [-150.99058171804361, -3946.626071010534, 5789.742898439815, -2.23046049968889, -5.931020059857665, -4.095581409074377]


The data shown here is the default GmatState vector data for a spacecraft.  The epoch is January 1, 2000 at 12:00:00.000 in TAIModJulian time, or 21545.00000039794 in A.1 ModJulian time.  Note that GMAT uses A.1 Mod Julian as its internal epoch system.  The state has 6 elements.  The position and velocity data are filled in with the dummy entries -999.999.
## Working with Cartesian and Keplerian Representations
A spacecraft in GMAT has a second collection of data: the state data for the spacecraft in the coordinate system set on the spacecraft.  These data are the spacecraft's "display state," named that way because they are the data displayed to the user.  Users interact with the display state similarly to the way they interact with the scripting language.  Data for a Keplerian state can be set using the SetField() method, as shown here:

In [14]:
sat.SetField("StateType", "Keplerian")
sat.SetField("SMA", 7015)
sat.SetField("ECC", 0.0011)
sat.SetField("INC", 98.6)
sat.SetField("RAAN", 75)
sat.SetField("AOP", 90)
sat.SetField("TA", 33.333)

At this point it can appear to the user that the data is set, but it really is not.  The spacecraft object cannot interpret the state data.  The data set using SetField needs more information than a spacecraft object can provide by itself.  Specifically, the spacecraft here does not have a connected coordinate system.  Cartesian state data set on the spacecraft does not have connections defining the coordinate origin, nor the structures needed to set the orientation of the axes defining directions.  Additionally, the spacecraft does not have the the gravitational constant needed to interpret Keplerian data.

In this uninitialized state, the spacecraft uses its GmatState buffer to hold the data entries.  We can see that the data is not yet fully populated by posting queries to the spacecraft:

In [15]:
print("The internal state buffer just holds preinitialization data (Keplerian here):\n  ", iState.GetState())
print("but access to the Keplerian state shows that it is not correct:\n  ", sat.GetKeplerianState())

The internal state buffer just holds preinitialization data (Keplerian here):
   [-150.990581718628, -3946.626071012089, 5789.742898438746, -2.2304604996888444, -5.9310200598564675, -4.095581409076133]
but access to the Keplerian state shows that it is not correct:
   7015.000000000009 0.001100000000000467 98.59999999999999 75               90.00000000002191 33.33299999999422


The Keplerian state data is not correct because the GMAT objects are not yet initialized.  Once we initialize the system, the Keplerian state will be correct, and the internal state will be updated to the EarthMJ2000Eq system.  The interobject connections necessary for these settings are made by calling the API's Initialize() function::

In [16]:
gmat.Initialize()
print("The initialized internal state buffer is EarthMJ2000Eq:\n  ", iState.GetState())
print("and the Keplerian state is correct:\n  ", sat.GetKeplerianState())

The initialized internal state buffer is EarthMJ2000Eq:
   [-150.990581718628, -3946.626071012089, 5789.742898438746, -2.2304604996888444, -5.9310200598564675, -4.095581409076133]
and the Keplerian state is correct:
   7015.000000000009 0.001100000000000467 98.59999999999999 75               90.00000000002191 33.33299999999422


Changes made to the state variables are now applied to the state as expected:

In [17]:
sat.SetField("SMA", 8000)
print("Internal state:\n ", iState.GetState())
print("Cartesian:\n ", sat.GetCartesianState())
print("Keplerian:\n ", sat.GetKeplerianState())
print()
sat.SetField("INC", 45)
print("Internal state:\n ", iState.GetState())
print("Cartesian:\n ", sat.GetCartesianState())
print("Keplerian:\n ", sat.GetKeplerianState())
print()
sat.SetField("TA", 50)
print("Internal state:\n ", iState.GetState())
print("Cartesian:\n ", sat.GetCartesianState())
print("Keplerian:\n ", sat.GetKeplerianState())

Internal state:
  [-172.1916826441942, -4500.785255608935, 6602.70038310904, -2.0886389885316343, -5.5539023177086415, -3.835168124651871]
Cartesian:
  -172.1916826441942 -4500.785255608935 6602.70038310904 -2.088638988531634 -5.553902317708642 -3.835168124651871
Keplerian:
  7999.999999999999 0.001100000000000386 98.59999999999999 75               90.00000000001269 33.33300000000347

Internal state:
  [-5697.741461949238, -3020.218654506183, 4721.905521454692, 1.120867903372707, -6.413887097496531, -2.7427113896956072]
Cartesian:
  -5697.741461949238 -3020.218654506183 4721.905521454692 1.120867903372707 -6.413887097496531 -2.742711389695607
Keplerian:
  7999.999999999999 0.001100000000000474 45               75.00000000000001 90.00000000000568 33.33300000001048

Internal state:
  [-5094.783427390105, -4974.90690274979, 3633.5822378219996, 2.516901195634427, -5.37973553882944, -3.8235178821450577]
Cartesian:
  -5094.783427390105 -4974.90690274979 3633.582237822   2.516901195634427 -5.

## Changing Coordinate Systems
The previous section shows how to access Cartesian and Keplerian representations of the system.  In this section we will work with a couple of different coordinate systems: an Earth fixed coordinate system named "ECF" and accessed using the Python reference ecf, and a solar ecliptic system named "SolarEcliptic," referenced as sec.  These coordinate systems are built using the code

In [18]:
ecf = gmat.Construct("CoordinateSystem", "ECF", "Earth", "BodyFixed")
sec = gmat.Construct("CoordinateSystem", "SolarEcliptic", "Sun", "MJ2000Ec")

In this section, the spacecraft sat defined previously will be used with the Earth fixed coordinate system, and a copy of that spacecraft will be used with the solar ecliptic system.  GMAT's objects support a method, Copy(), that copies one object into another object of the same type.  Rather than set up a new spacecraft from scratch, we'll use that framework to get started by creating a new spacecraft and then setting the coordinate systems so that the original spacecraft uses the ECI coordinate system and the new spacecraft uses the solar ecliptic system.

In [19]:
solsat = gmat.Construct("Spacecraft","SolarSat")
solsat.Copy(sat)

# Now set coordinate systems
sat.SetField("CoordinateSystem","ECF")
solsat.SetField("CoordinateSystem","SolarEcliptic")

We've reset the coordinate system names on the spacecraft at this point, but have yet to reset the associated objects because the Initialize() function that connects objects together has not been called since making the reassignment.  The data reflects this state of the system:

In [20]:
# Show the data after setting the new coordinate systems, before initialization
print("The spacecraft ", sat.GetName(), " initialization state is ", sat.IsInitialized())
print("The internal state buffer:  ", iState.GetState())
print("The ECF Cartesian State:    ", sat.GetCartesianState())
print("The ECF Keplerian State:    ", sat.GetKeplerianState())
print()
print("The spacecraft ", solsat.GetName(), " initialization state is ", sat.IsInitialized())
print("The internal state buffer (SolarSat):  ", solsat.GetState().GetState())
print("The SolarEcliptic Cartesian State:     ", solsat.GetCartesianState())
print("The SolarEcliptic Keplerian State:     ", solsat.GetKeplerianState())

The spacecraft  MySat  initialization state is  True
The internal state buffer:   [-5094.783427390105, -4974.90690274979, 3633.5822378219996, 2.516901195634427, -5.37973553882944, -3.8235178821450577]
The ECF Cartesian State:     -5094.783427390105 -4974.90690274979 3633.582237822   2.516901195634427 -5.37973553882944 -3.823517882145058
The ECF Keplerian State:     8000.000000000003 0.001100000000000762 44.99999999999999 75               90.00000000001448 49.99999999999122

The spacecraft  SolarSat  initialization state is  True
The internal state buffer (SolarSat):   [-5094.783427390105, -4974.90690274979, 3633.5822378219996, 2.516901195634427, -5.37973553882944, -3.8235178821450577]
The SolarEcliptic Cartesian State:      -5094.783427390105 -4974.90690274979 3633.582237822   2.516901195634427 -5.37973553882944 -3.823517882145058
The SolarEcliptic Keplerian State:      8000.000000000003 0.001100000000000762 44.99999999999999 75               90.00000000001448 49.99999999999122


*Note that the initialization state reported here is a bug: resetting object references should toggle the initialization flag, but did not.*

Once we initialize the system, replacing the coordinate system references with the correct objects, the data is once again correct:

In [21]:
# Connect the GMAT objects together
gmat.Initialize()

# And show the data in the new coordinate systems
print("The internal state buffer:  ", iState.GetState())
print("The ECF Cartesian State:    ", sat.GetCartesianState())
print("The ECF Keplerian State:    ", sat.GetKeplerianState())
print()
print("The internal state buffer (SolarSat):  ", solsat.GetState().GetState())
print("The SolarEcliptic Cartesian State:     ", solsat.GetCartesianState())
print("The SolarEcliptic Keplerian State:     ", solsat.GetKeplerianState())

The internal state buffer:   [-5094.783427390105, -4974.90690274979, 3633.5822378219996, 2.516901195634427, -5.37973553882944, -3.8235178821450577]
The ECF Cartesian State:     3980.769665921791 -5904.072174048258 3633.846635805904 5.313373701952485 1.22119013772983 -3.823433741949215
The ECF Keplerian State:     7197.708272712499 0.1106817774544233 47.10837940069969 152.2889390195662 322.0637563060873 179.5887814511723

The internal state buffer (SolarSat):   [-5094.783427390105, -4974.90690274979, 3633.5822378219996, 2.516901195634427, -5.37973553882944, -3.8235178821450577]
The SolarEcliptic Cartesian State:      -26505087.9080278 144694001.6268158 4700.44201990217 -27.27732399951875 -11.92620879192174 -1.367891168159899
The SolarEcliptic Keplerian State:      144849901.113105 0.2292154440704457 2.702016602263789 280.4191667873195 286.9680459338969 252.9931176051899
